In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import simps, trapz

In [2]:
meta_df = pd.read_csv('metadata.csv')

In [3]:
battery_id_df = meta_df['battery_id'].unique()
battery_id_df

array(['B0047', 'B0045', 'B0048', 'B0046', 'B0043', 'B0032', 'B0039',
       'B0040', 'B0029', 'B0028', 'B0042', 'B0034', 'B0038', 'B0033',
       'B0030', 'B0041', 'B0027', 'B0044', 'B0036', 'B0025', 'B0026',
       'B0031', 'B0049', 'B0050', 'B0052', 'B0051', 'B0006', 'B0005',
       'B0007', 'B0018', 'B0053', 'B0054', 'B0056', 'B0055'], dtype=object)

In [29]:
list(battery_id_df).index('B0028')

9

In [37]:
battery_id_df[9]

'B0028'

In [36]:
battery_df = meta_df[(meta_df.battery_id == battery_id_df[9]) & (meta_df.type != 'impedance')]['type']
battery_df.values

array(['charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge'], dtype=object)

In [32]:
for battery_id in battery_id_df[9:10]:
    battery_df = meta_df[(meta_df.battery_id == battery_id) & (meta_df.type != 'impedance')]
    battery_df.reset_index(drop=True, inplace=True)

    vit_df = pd.DataFrame()
    time = 0    
    for idx, detail in battery_df.iterrows():
        if idx % 4 == 0 and idx != 0:
            vit_df['Time'] /= 3600
            vit_df.to_csv('./bms/' + "SDINR" + str(battery_id) + "DX" + str(idx)  + '.csv', index=False)
            vit_df = pd.DataFrame()
            time = 0
       
        temp_df = pd.read_csv('./log/' +  battery_df.iloc[idx]['filename'])
        
        if detail['type'] == 'discharge':
            temp_df = temp_df[temp_df['Current_load'] > 0.1].reset_index(drop=True)
        elif detail['type'] == 'charge':
            temp_df = temp_df[temp_df.index >= 2].reset_index(drop=True)

        temp_df['Time'] += time        
        time = temp_df['Time'][-1:].iloc[0] + 10
        # print(temp_df['Time'][-1:].iloc[0])

        temp_df = temp_df[temp_df.columns[~temp_df.columns.isin(['Current_load', 'Voltage_load', 'Current_charge', 'Voltage_charge'])]]
        vit_df = pd.concat([vit_df, temp_df], ignore_index=True)

In [11]:
test_df = pd.read_csv('./bms/SDINRB0029DX60.csv')
test_df

,Voltage_measured,Current_measured,Temperature_measured,Time
0,3.884615,-4.024371,44.720955,0.005759
1,3.856344,-4.024532,44.819976,0.008720
2,3.834882,-4.022963,44.985736,0.011662
3,3.816247,-4.025845,45.192361,0.014640
4,3.799913,-4.024016,45.402604,0.017596
...,...,...,...,...
4835,4.199862,-0.002757,44.685552,5.995755
4836,4.199937,-0.000168,44.691869,5.996893
4837,4.199914,-0.001453,44.674629,5.998034
4838,4.199959,0.000575,44.675299,5.999149


In [12]:
test_df[test_df['Temperature_measured']> 50]

,Voltage_measured,Current_measured,Temperature_measured,Time
34,3.558026,-4.023310,50.074175,0.106371
35,3.552136,-4.023019,50.204887,0.109345
36,3.546262,-4.023496,50.296228,0.112300
37,3.540680,-4.024868,50.395800,0.115256
38,3.534907,-4.024215,50.477349,0.118207
...,...,...,...,...
2551,2.807685,-5.022108,58.497399,3.397222
2552,2.702767,-4.025555,58.736070,3.400230
2553,2.551198,-4.025704,58.901884,3.403234
2554,2.310506,-4.021428,59.210780,3.406228


In [229]:
# 쿨롱 계수 계산
def coulomb_integral(time, current, method='trapz'):
    if method == 'trapz':
        capacity = np.abs(trapz(current, time))  # 사다리꼴 방법
    elif method == 'simps':
        capacity = np.abs(simps(current, time))  # 심프슨 방법
    else:
        raise ValueError('Invalid method')
    return capacity


In [238]:
num = [1,2,3,4]
num = [x / 5 for x in num]
num

[0.2, 0.4, 0.6, 0.8]

In [239]:
def discharge_test(idx, time):
    # 방전 데이터
    discharge_df = pd.read_csv('./log/' +  experiment_df.iloc[idx]['filename'])
    discharge_df['Time'] += time
    discharge_df

    time = discharge_df['Time'].values # 초
    time = [x / 3600 for x in time]
    current = discharge_df['Current_measured'].values # 전류

    # 구간별 쿨롱 계수 계산
    discharge_df['coulomb'] = 0.0  # 쿨롱 계수

    for i in discharge_df.index:
        time = discharge_df['Time'][:i+1].to_numpy()
        current = discharge_df['Current_measured'][:i+1].to_numpy()

        capacity_simps = coulomb_integral(time, current, method='simps')
        discharge_df.loc[i, 'coulomb'] = capacity_simps

    SOC_0 = discharge_df['coulomb'][-1:].iloc[0]
    discharge_df['SOC'] = ((SOC_0 - discharge_df['coulomb']) / SOC_0) * 100

    return discharge_df

In [240]:
def charge_test(idx, time):
    # 충전 데이터
    charge_df = pd.read_csv('./log/' +  experiment_df.iloc[idx]['filename'])
    charge_df = charge_df[charge_df['Current_charge'] >= 0]
    charge_df['Time'] += time
    charge_df

    time = charge_df['Time'].values # 초
    time = [x / 3600 for x in time]
    current = charge_df['Current_measured'].values # 전류

    # 구간별 쿨롱 계수 계산
    charge_df['coulomb'] = 0.0  # 쿨롱 계수

    for i in charge_df.index:
        time = charge_df['Time'][:i+1].to_numpy()
        current = charge_df['Current_measured'][:i+1].to_numpy()
        
        capacity_simps = coulomb_integral(time, current, method='simps')
        charge_df.loc[i, 'coulomb'] = capacity_simps

    SOC_0 = charge_df['coulomb'][-1:].iloc[0]
    charge_df['SOC'] = (charge_df['coulomb'] / SOC_0) * 100
    
    return charge_df

In [241]:
test_df = pd.DataFrame()

time = 0
for idx, experiment in experiment_df.iterrows():
    if experiment['type'] == 'discharge':
        df = discharge_test(idx , time)
    elif experiment['type'] == 'charge':
        df = charge_test(idx , time)
    time = df['Time'][-1:].iloc[0] + 10
    
    test_df = pd.concat([test_df, df], ignore_index=True)
test_df

,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Time,coulomb,SOC,Current_charge,Voltage_charge
0,4.246711,0.000252,6.212696,0.0002,0.000,0.000,0.000000,100.000000,NaN,NaN
1,4.246764,-0.001411,6.234019,0.0002,4.262,9.360,0.005423,99.999912,NaN,NaN
2,4.039277,-0.995093,6.250255,1.0000,3.465,23.281,5.148440,99.916112,NaN,NaN
3,4.019506,-0.996731,6.302176,1.0000,3.451,36.406,19.212586,99.686951,NaN,NaN
4,4.004763,-0.992845,6.361645,1.0000,3.438,49.625,31.381999,99.488664,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
125321,4.211567,0.062552,5.930646,NaN,NaN,1124890.475,4235.871363,99.961859,0.0632,4.263
125322,4.211716,0.061251,5.938026,NaN,NaN,1124899.162,4236.407512,99.974511,0.0632,4.263
125323,4.211562,0.063422,5.960594,NaN,NaN,1124907.834,4236.956970,99.987478,0.0632,4.263
125324,4.211579,0.061475,5.963431,NaN,NaN,1124916.584,4237.487593,100.000000,0.0632,4.263


In [13]:
# 4.2보다 큰 값을 가지는 행을 찾는다.
condition = test_df['Voltage_measured'] > 4.2

test_df['group'] = (condition != condition.shift()).cumsum()

# 조건을 만족하는 행들만 필터링한다.
filtered_df = test_df[condition]
filtered_df

# 그룹 번호로 그룹화한다
grouped = filtered_df.groupby('group')

# 첫 번째 행의 인덱스를 구한다.
first_index = grouped.head(1).index

# 마지막 행의 인덱스를 구한다.
last_index = grouped.tail(1).index

# 인덱스들을 데이터 프레임으로 변환한다.
index_df = pd.DataFrame({'first_index': first_index, 'last_index': last_index}).reset_index(drop=True)

# 시간의 최소값 최대값의 index를 추출한다.
# index_df = grouped.agg(first_index=('Time', 'idxmin'), last_index=('Time', 'idxmax'))

index_df

,first_index,last_index
0,970,2116
1,2120,2412
2,3369,4496
3,4501,4797
4,4799,4799
5,4801,4804
6,4806,4813
7,4815,4815
8,4817,4817
9,4821,4821


In [15]:
# 4.2보다 큰 값을 가지는 행을 찾는다.
condition = test_df['Voltage_measured'] < 2.8

test_df['group'] = (condition != condition.shift()).cumsum()

# 조건을 만족하는 행들만 필터링한다.
filtered_df = test_df[condition]
filtered_df

# 그룹 번호로 그룹화한다
grouped = filtered_df.groupby('group')

# 첫 번째 행의 인덱스를 구한다.
first_index = grouped.head(1).index

# 마지막 행의 인덱스를 구한다.
last_index = grouped.tail(1).index

# 인덱스들을 데이터 프레임으로 변환한다.
index_df = pd.DataFrame({'first_index': first_index, 'last_index': last_index}).reset_index(drop=True)

index_df

,first_index,last_index
0,140,143
1,2552,2555


In [362]:
temp_df = meta_df[(meta_df.battery_id == battery_id_df[12]) & (meta_df.type != 'impedance')]
temp_df.reset_index(drop=True, inplace=True)
temp_df

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,discharge,[2010. 4. 14. 16. 43. ...,24,B0038,0,2291,02291.csv,0.8980573709676724,NaN,NaN
1,charge,[2010. 4. 14. 17. 37. ...,24,B0038,2,2293,02293.csv,NaN,NaN,NaN
2,discharge,[2010. 4. 14. 21. 10. ...,24,B0038,4,2295,02295.csv,1.1026236331914077,NaN,NaN
3,charge,[2010. 4. 14. 21. 36. ...,24,B0038,5,2296,02296.csv,NaN,NaN,NaN
4,discharge,[2010. 4. 15. 0. 37. ...,24,B0038,6,2297,02297.csv,1.0885801879473431,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
89,charge,[2010. 4. 29. 9. 40. ...,44,B0038,114,2405,02405.csv,NaN,NaN,NaN
90,discharge,[2.010e+03 5.000e+00 4.000e+00 1.300e+01 2.200...,44,B0038,116,2407,02407.csv,1.52573884649634,NaN,NaN
91,charge,[2010. 5. 4. 14. 24. ...,44,B0038,118,2409,02409.csv,NaN,NaN,NaN
92,discharge,[2010. 5. 4. 17. 26. ...,44,B0038,120,2411,02411.csv,1.5301475534434217,NaN,NaN


In [369]:
temp_df

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,discharge,[2010. 4. 14. 16. 43. ...,24,B0038,0,2291,02291.csv,0.8980573709676724,NaN,NaN
1,charge,[2010. 4. 14. 17. 37. ...,24,B0038,2,2293,02293.csv,NaN,NaN,NaN
2,discharge,[2010. 4. 14. 21. 10. ...,24,B0038,4,2295,02295.csv,1.1026236331914077,NaN,NaN
3,charge,[2010. 4. 14. 21. 36. ...,24,B0038,5,2296,02296.csv,NaN,NaN,NaN
4,discharge,[2010. 4. 15. 0. 37. ...,24,B0038,6,2297,02297.csv,1.0885801879473431,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
89,charge,[2010. 4. 29. 9. 40. ...,44,B0038,114,2405,02405.csv,NaN,NaN,NaN
90,discharge,[2.010e+03 5.000e+00 4.000e+00 1.300e+01 2.200...,44,B0038,116,2407,02407.csv,1.52573884649634,NaN,NaN
91,charge,[2010. 5. 4. 14. 24. ...,44,B0038,118,2409,02409.csv,NaN,NaN,NaN
92,discharge,[2010. 5. 4. 17. 26. ...,44,B0038,120,2411,02411.csv,1.5301475534434217,NaN,NaN


In [376]:
# 일단 38번
# 최대 capacity

initial_soc = 100  # 초기 SOC 값을 100%로 설정
initial_capacity = 1.8053927676991701 # 최대 capacity

# 쿨롱 계수를 활용한 SOC 계산
def soc_estimation(df):

    capacity_li = []

    for i, ex in temp_df.iterrows():
        if experiment['type'] == 'discharge':
            # 방전 데이터
            discharge_df = pd.read_csv('./log/' +  df.iloc[idx]['filename'])
    
            discharge_df['capacity'] = 0.0
            discharge_df['Time'] /= 3600

            for i in df.index:
                time = discharge_df['Time'][:i+1].to_numpy()
                current = discharge_df['Current_measured'][:i+1].to_numpy()
                capacity = np.abs(simps(current, time))
                discharge_df.loc[i, 'capacity'] = capacity
            ca = discharge_df.iloc[-1:]['capacity']
            capacity_li.append(ca)

    return capacity_li

li = soc_estimation(temp_df)
li

[]

In [ ]:
import numpy as np
from scipy.integrate import simps

initial_soc = 100  # 초기 SOC 값을 100%로 설정

# 쿨롱 계수를 활용한 SOC 계산
def soc_estimation(time, current, initial_soc):
    capacity = np.abs(simps(current, time))  # 쿨롱 계수 계산
    soc = initial_soc - (capacity / initial_capacity * 100)  # SOC 업데이트
    return max(0, min(soc, 100))  # SOC가 0과 100 사이의 값이 되도록 조정

estimated_soc = soc_estimation(time, current, initial_soc)
print('Estimated SOC: {:.2f}%'.format(estimated_soc))


In [423]:
discharge_df = pd.read_csv('./log/' +  temp_df.iloc[30]['filename'])

initial_soc = 100  # 초기 SOC 값을 100%로 설정
initial_capacity = 1.8053927676991701 # 최대 capacity

discharge_df['Time'] /= 3600
time = discharge_df['Time'].values
current = discharge_df['Current_measured'].values
discharge_df['soc'] = 0.0

for i in discharge_df.index:
    time = discharge_df['Time'][:i+1].to_numpy()
    current = discharge_df['Current_measured'][:i+1].to_numpy()

    capacity_simps = coulomb_integral(time, current, method='simps')
    capacity = np.abs(simps(current, time))  # 쿨롱 계수 계산

    soc = initial_soc - (capacity / initial_capacity * 100)  # SOC 업데이트
    discharge_df.loc[i, 'soc'] = max(0, min(soc, 100))  # SOC가 0과 100 사이의 값이 되도록 조정

discharge_df

,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Time,soc
0,4.189402,-0.001149,45.370673,0.0004,0.000,0.000000,100.000000
1,4.189260,-0.002653,45.393008,0.0004,4.205,0.002591,99.999727
2,4.031560,-1.004874,45.393880,0.9986,3.543,0.005647,99.939050
3,4.025193,-1.006782,45.391874,0.9986,3.539,0.008472,99.770041
4,4.020052,-1.008072,45.420474,0.9986,3.535,0.011311,99.623218
...,...,...,...,...,...,...,...
641,2.690464,-0.004231,48.911061,0.0004,0.000,1.812943,0.002017
642,2.702761,-0.001707,48.865222,0.0004,0.000,1.815877,0.001447
643,2.713018,-0.002479,48.798921,0.0004,0.000,1.818854,0.001148
644,2.721699,-0.003621,48.733140,0.0004,0.000,1.821797,0.000615


In [416]:
temp_df = meta_df[(meta_df.battery_id == battery_id_df[12]) & (meta_df.type != 'impedance')]
temp_df.reset_index(drop=True, inplace=True)
temp_df

li = []
for i, ex in temp_df.iterrows():
    if ex['type'] == 'discharge':
            # 방전 데이터
            discharge_df = pd.read_csv('./log/' +  temp_df.iloc[i]['filename'])

            discharge_df['Time'] /= 3600
            time = discharge_df['Time'].values
            current = discharge_df['Current_measured'].values
            capacity = np.abs(simps(current, time))
            li.append((i,capacity))

In [ ]:
def discharge_test(df, idx, time, SOC_0):
    # 방전 데이터
    discharge_df = pd.read_csv('./log/' +  df.iloc[idx]['filename'])
    discharge_df['Time'] += time
    discharge_df

    # 구간별 쿨롱 계수 계산
    discharge_df['coulomb'] = 0.0  # 쿨롱 계수

    for i in discharge_df.index:
        time = discharge_df['Time'][:i+1].to_numpy()
        current = discharge_df['Current_measured'][:i+1].to_numpy()

        capacity_simps = coulomb_integral(time, current, method='simps')
        discharge_df.loc[i, 'coulomb'] = capacity_simps

    discharge_df['SOC'] = ((SOC_0 - discharge_df['coulomb']) / SOC_0) * 100

    return discharge_df

In [ ]:
def charge_test(df, idx, time):
    # 충전 데이터
    charge_df = pd.read_csv('./log/' +  df.iloc[idx]['filename'])
    charge_df['Time'] += time
    charge_df

    time = charge_df['Time'].values # 초
    time = [x / 3600 for x in time]
    current = charge_df['Current_measured'].values # 전류

    # 구간별 쿨롱 계수 계산
    charge_df['coulomb'] = 0.0  # 쿨롱 계수

    for i in charge_df.index:
        time = charge_df['Time'][:i+1].to_numpy()
        current = charge_df['Current_measured'][:i+1].to_numpy()
        
        capacity_simps = coulomb_integral(time, current, method='simps')
        charge_df.loc[i, 'coulomb'] = capacity_simps

    charge_df['SOC'] = (charge_df['coulomb'] / SOC_0) * 100
    
    return charge_df

In [309]:
temp_df = meta_df[(meta_df.battery_id == battery_id_df[12]) & (meta_df.type != 'impedance')]
temp_df.reset_index(drop=True, inplace=True)

time = 0
test_temp_df = pd.DataFrame()
for i, ex in temp_df.iterrows():
    if experiment['type'] == 'discharge':
        t_df = discharge_test(temp_df, idx , time)
    elif experiment['type'] == 'charge':
        t_df = charge_test(temp_df, idx , time)
    time = t_df['Time'][-1:].iloc[0] + 10

    test_temp_df = pd.concat([test_temp_df, t_df], ignore_index=True)
test_temp_df

,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Time,Current_charge,Voltage_charge
0,4.053117,-0.003010,25.045829,0.0002,0.000,0.000,NaN,NaN
1,4.052972,-0.004830,25.117199,0.0002,4.068,9.360,NaN,NaN
2,3.330114,-4.029557,25.297835,4.0010,1.400,19.516,NaN,NaN
3,3.274633,-4.029148,25.550915,4.0010,1.345,28.875,NaN,NaN
4,3.225083,-4.028353,25.837273,4.0010,1.291,38.219,NaN,NaN
...,...,...,...,...,...,...,...,...
138418,4.201415,0.956716,47.643312,NaN,NaN,3151.968,0.950,4.716
138419,4.201981,0.955189,47.629235,NaN,NaN,3157.125,0.950,4.716
138420,4.201102,0.946626,47.638965,NaN,NaN,3162.250,0.940,4.710
138421,NaN,NaN,NaN,NaN,NaN,3166.484,0.940,4.710


In [226]:
temp_df = meta_df[(meta_df.battery_id == battery_id_df[12]) & (meta_df.type != 'impedance')]
temp_df.reset_index(drop=True, inplace=True)

test_temp_df = pd.DataFrame()
for i, ex in temp_df.iterrows():
    t_df = pd.read_csv('./log/' +  temp_df.iloc[i]['filename'])
    test_temp_df = pd.concat([test_temp_df, t_df], ignore_index=True)
test_temp_df

,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Time,Current_charge,Voltage_charge
0,4.053117,-0.003010,25.045829,0.0002,0.000,0.000,NaN,NaN
1,4.052972,-0.004830,25.117199,0.0002,4.068,9.360,NaN,NaN
2,3.330114,-4.029557,25.297835,4.0010,1.400,19.516,NaN,NaN
3,3.274633,-4.029148,25.550915,4.0010,1.345,28.875,NaN,NaN
4,3.225083,-4.028353,25.837273,4.0010,1.291,38.219,NaN,NaN
...,...,...,...,...,...,...,...,...
138418,4.201415,0.956716,47.643312,NaN,NaN,3151.968,0.950,4.716
138419,4.201981,0.955189,47.629235,NaN,NaN,3157.125,0.950,4.716
138420,4.201102,0.946626,47.638965,NaN,NaN,3162.250,0.940,4.710
138421,NaN,NaN,NaN,NaN,NaN,3166.484,0.940,4.710


In [9]:
df = pd.DataFrame(battery_id_df)

In [12]:
df.to_csv('./vits/bat.csv', index=False)

In [33]:
battery_id_df[20]

'B0026'

### 각각의 배터리 셀 데이터

In [11]:
for battery_id in battery_id_df[20:21]:
    battery_df = meta_df[(meta_df.battery_id == battery_id) & (meta_df.type != 'impedance')]['type']
battery_df.values

array(['charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge'], dtype=object)

In [9]:
for battery_id in battery_id_df[19:20]:
    battery_df = meta_df[(meta_df.battery_id == battery_id) & (meta_df.type != 'impedance')]['type']
battery_df.values

array(['charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge', 'charge',
       'discharge', 'charge', 'discharge', 'charge', 'discharge',
       'charge', 'discharge', 'charge', 'discharge', 'charge',
       'discharge', 'charge'], dtype=object)

In [29]:
bat_df = pd.read_csv("./bms/B0025_4.csv")
bat_df

,Voltage_measured,Current_measured,Temperature_measured,Time
0,3.210795,-0.001231,26.935337,0.000000
1,2.802418,-3.040619,26.936643,0.000699
2,3.371466,1.508858,26.960100,0.001446
3,3.395449,1.510285,26.976026,0.002131
4,3.411504,1.509259,26.982651,0.002817
...,...,...,...,...
7882,2.689351,-4.026231,40.375791,0.920881
7883,2.590396,-4.025844,40.524338,0.926458
7884,2.438817,-4.027036,40.710165,0.932088
7885,2.185966,-4.025640,41.007300,0.937739


In [30]:
bat_df[bat_df['Time'] > 0.9]

,Voltage_measured,Current_measured,Temperature_measured,Time
1271,4.206933,1.406863,29.867731,0.900447
1272,4.206298,1.395181,29.871530,0.901294
1273,4.205651,1.386562,29.875320,0.902127
1274,4.205835,1.384593,29.875594,0.902960
1275,4.206001,1.387895,29.882945,0.903772
...,...,...,...,...
7882,2.689351,-4.026231,40.375791,0.920881
7883,2.590396,-4.025844,40.524338,0.926458
7884,2.438817,-4.027036,40.710165,0.932088
7885,2.185966,-4.025640,41.007300,0.937739


In [58]:
vit_df.to_csv('./vits/' + battery_id + '.csv', index=False)

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,discharge,[2010. 7. 21. 15. 0. ...,4,B0047,0,1,00001.csv,1.6743047446975208,NaN,NaN
1,charge,[2010. 7. 21. 17. 25. ...,4,B0047,2,3,00003.csv,NaN,NaN,NaN
2,discharge,[2.0100e+03 7.0000e+00 2.1000e+01 2.1000e+01 2...,4,B0047,4,5,00005.csv,1.5243662105099023,NaN,NaN
3,charge,[2010. 7. 21. 22. 38. ...,4,B0047,5,6,00006.csv,NaN,NaN,NaN
4,discharge,[2.010e+03 7.000e+00 2.200e+01 1.000e+00 4.000...,4,B0047,6,7,00007.csv,1.5080762969973425,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
139,charge,[2010 8 17 2 32 20],4,B0047,177,178,00178.csv,NaN,NaN,NaN
140,discharge,[2010. 8. 17. 5. 33. ...,4,B0047,178,179,00179.csv,1.158443027293818,NaN,NaN
141,charge,[2010. 8. 17. 6. 50. ...,4,B0047,179,180,00180.csv,NaN,NaN,NaN
142,discharge,[2010. 8. 17. 9. 52. ...,4,B0047,180,181,00181.csv,1.1567087516841796,NaN,NaN


In [47]:
temp_df['Time'][-1:].iloc[0] + 10

6446.141

In [98]:
battery_df = meta_df[(meta_df.battery_id == 'B0052') & (meta_df.type != 'impedance')]

In [97]:
battery_id_df[23]

'B0050'

In [65]:
t_charge_df = pd.read_csv('./log/' +  battery_df.iloc[59]['filename'])
t_charge_df

,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time
0,3.557399,-0.003034,7.773916,-0.0006,0.002,0.000
1,3.813916,1.487826,7.758312,1.4995,4.707,2.500
2,3.856283,1.490303,7.737755,1.4995,4.745,9.718
3,3.883881,1.489862,7.704689,1.4995,4.757,16.875
4,3.906531,1.489249,7.714172,1.4995,4.757,24.047
...,...,...,...,...,...,...
1419,4.212304,0.061660,5.168480,0.0621,4.263,10771.468
1420,4.212553,0.063637,5.177620,0.0626,4.263,10779.140
1421,4.212328,0.060296,5.191178,0.0626,4.263,10786.781
1422,4.212469,0.061967,5.217356,0.0626,4.263,10794.390


In [51]:
t_discharge_df = pd.read_csv('./log/' +  battery_df.iloc[0]['filename'])
t_discharge_df = temp_df[temp_df['Current_load'] > 0.1].reset_index(drop=True)
t_discharge_df

,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Time
0,4.039277,-0.995093,6.250255,1.0,3.465,23.281
1,4.019506,-0.996731,6.302176,1.0,3.451,36.406
2,4.004763,-0.992845,6.361645,1.0,3.438,49.625
3,3.992863,-0.995957,6.399692,1.0,3.427,62.813
4,3.982347,-0.994665,6.233613,1.0,3.419,75.891
...,...,...,...,...,...,...
464,2.593520,-0.993901,11.977146,1.0,2.052,6125.438
465,2.565613,-0.995725,12.038215,1.0,2.025,6138.875
466,2.536253,-0.993844,12.128629,1.0,1.996,6152.281
467,2.504669,-0.994458,12.234626,1.0,1.965,6165.672


In [30]:
temp_df['Time']

Series([], Name: Time, dtype: float64)

In [225]:
temp_df = meta_df[(meta_df.battery_id == battery_id_df[13]) & (meta_df.type != 'impedance')]
temp_df.reset_index(drop=True, inplace=True)

test_temp_df = pd.DataFrame()
for i, ex in temp_df.iterrows():
    t_df = pd.read_csv('./log/' +  temp_df.iloc[i]['filename'])
    test_temp_df = pd.concat([test_temp_df, t_df], ignore_index=True)
test_temp_df

,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Time,Current_charge,Voltage_charge
0,3.677219,-0.001741,23.712130,0.0002,0.000,0.000,NaN,NaN
1,3.676917,-0.001644,23.709047,0.0000,3.690,9.328,NaN,NaN
2,2.914905,-4.026855,23.834783,3.9950,0.937,19.421,NaN,NaN
3,2.871878,-4.024270,24.249977,3.9950,0.904,28.750,NaN,NaN
4,2.833588,-4.026631,24.806260,3.9950,0.861,38.125,NaN,NaN
...,...,...,...,...,...,...,...,...
406807,4.206106,0.032829,24.106851,NaN,NaN,10781.859,0.034,4.24
406808,4.205883,0.034818,24.099885,NaN,NaN,10790.968,0.034,4.24
406809,4.205776,0.030391,24.101408,NaN,NaN,10800.000,0.034,4.24
406810,4.205773,0.033583,24.095999,NaN,NaN,10809.046,0.034,4.24
